<h1 style="text-align: center;">Seminario N°4:</h1>
<h1 style="text-align: center;">Construcción y análisis de sets de datos para el desarrollo de modelos de QSAR</h1>


## 🎯  Objetivos

* Conocer el manejo de bases de datos disponibles para poder llevar cabo un análisis sistemático de relaciones estructura-actividad (SAR).
* Aplicar los conocimientos adquiridos en las actividades previas (Sem. y TP N°1 a 3) a un caso de estudio concreto y clínicamente relevante.
* Generar un conjunto de datos a ser utilizado en el TP N°4 para construir modelos de QSAR.

## 🔍 Introducción a las bases de datos químicas en QSAR

El desarrollo de modelos de relaciones cuantitativas estructura-actividad (QSAR) requiere contar con **conjuntos de datos curados, estructurados y relevantes**, tanto desde el punto de vista químico como biológico. Afortunadamente, existen numerosas **bases de datos públicas** que centralizan información sobre compuestos bioactivos, sus estructuras químicas, propiedades fisicoquímicas y actividades biológicas medidas experimentalmente.

Entre las más utilizadas en química medicinal y para el diseño de fármacos, se destacan:

- ***[PubChem](https://pubchem.ncbi.nlm.nih.gov/)***: a cargo del National Center for Biotechnology Information (NCBI); contiene millones de compuestos, bioensayos y datos experimentales.
- ***[ChEMBL](https://www.ebi.ac.uk/chembl/)***: a cargo del European Molecular Biology Laboratory del European Bioinformatics Institute (EMBL-EBI), con datos extraídos de literatura científica.
- ***[DrugBank](https://go.drugbank.com/)***, **BindingDB**, **ZINC**, **PDB**, entre otras.

Cada una tiene características particulares, pero en esta actividad emplearemos ChEMBL: https://www.ebi.ac.uk/chembl/.


---

## 📦 ¿Qué es ChEMBL?

**ChEMBL** es una base de datos de bioactividad de compuestos químicos con interés farmacéutico/farmacológico, generada y mantenida por el EMBL-EBI. Algunos tipos de datos que reúne son:

- Información estructural y propiedades biofarmacéuticas (SMILES, InChI, peso molecular, logP, etc.) de compuestos.
- Información farmacológica (mecanismo de acción, fase clínica, nombre comercial).
- Actividades biológicas (IC₅₀, Kᵢ, EC₅₀, etc.) contra blancos moleculares definidos.
- Ensayos experimentales reportados en literatura, con conexión directa a la bibliografía.

<div align="center">
<img src="ChEMBL.png" alt="Figura 1" width="600"/>
</div>
    
Los compuestos están organizados en torno a **blancos terapéuticos** (enzimas, receptores, transportadores) y pueden tener distintas **acciones farmacológicas** como agonistas, antagonistas, inhibidores u otros mecanismos.

**ChEMBL** se encuentra construída sobre la base de los que se conoce como *bases de datos estructuradas*, permitiendo realizar búsquedas conectando una gran cantidad de información como criterio de filtrado de resultados. La imagen que sigue muestra un plano general de cómo está estructurada la información de millones de moléculas:

<div align="center">
  <img src="ChEMBL_schema.png" alt="Figura 999" width="1200"/>
</div>


Esto la convierte en una **fuente valiosa para desarrollar modelos QSAR**, ya que ofrece datos confiables y bien clasificados, fundamentales para correlacionar estructuras, propiedades y actividad biológica.

---

## 🎯 ¿Qué haremos en este seminario?

En esta clase aprenderemos a:

1. **Buscar, filtrar y compilar datos desde ChEMBL**, seleccionando un blanco biológico clínicamente relevante.
2. **Procesar y limpiar los datos**: por ejemplo, eliminar duplicados, entradas inválidas o incompletas.
3. **Visualizar estructuras y propiedades moleculares**, peparando los datos para su posterior análisis.
4. **Explorar/analizar/delimitar el espacio químico de la serie de análogos en estudio**, con el objeto de evaluar diferencias y semejanzas estructurales y de propiedades. Dicho análisis de espacio químico permitirá además delimitar un *dominio de aplicación* de los modelos de QSAR resultantes.
5. **Construir un conjunto de datos depurados (dataset)** para su uso posterior en modelos QSAR (TP N°4).

---


### 🔧 Instalación de ChEMBL en Python

Para poder acceder a la base de datos **ChEMBL** sin necesidad de trabajar en la página web, vamos a utilizar un **módulo oficial** que nos permite hacer consultas directas usando funciones simples.

Podemos instalarlo ejecutando la siguiente celda de código:

In [ ]:
!pip install -q chembl_webresource_client

Además, como lo hemos hecho siempre, vamos a importar algunas **funciones específicas** que hemos preparado para esta clase a través del siguiente código:

In [ ]:
!pip install -q numpy==1.24.4
!pip install -q chemplot
from FuncionesSem4 import *

### 🔎 Búsqueda de información sobre un fármaco conocido

La siguiente función permite buscar compuestos en ChEMBL utilizando su nombre comercial o genérico, y devuelve **toda** la información disponible para cada molécula.


In [ ]:
df_farmaco = buscar_info_nombre_comercial("propranolol")
itables.show(df_farmaco)

### 🧬 Buscar información sobre un blanco terapéutico conocido

Además de fármacos, podríamos desear conocer información sobre blancos terapéuticos. La siguiente función permite buscar dichos datos en ChEMBL utilizando una palabra clave de receptores, enzimas, canales, etc., y devuelve **toda** la información asociada. 


In [ ]:
df_target = buscar_info_target("adrenergic")
itables.show(df_target)

Ahora que ya conocemos a grandes rasgos cómo acceder y qué información podemos obtener de ChEMBL, podemos enfocarnos en un caso clínico concreto: **fármacos adrenérgicos**.

<h1 style="text-align: center;">Caso de estudio: Fármacos adrenérgicos</h1>

Los **receptores adrenérgicos** (RA) son proteínas de membrana que median las acciones producidas por los neurotransmisores del sistema nervioso central *epinefrina* y *norepinefrina*, también llamados como adrenalina y noradrenalina, respectivamente. 

<div align="center">
<img src="EpiNorepi_Interac.png" alt="Figura 2" width="800"/>
<div\>
    
Estos receptores pertenecen a la familia de los **receptores acoplados a proteínas G (GPCRs)**, y se dividen en dos grandes subtipos: **α-adrenérgicos** y **β-adrenérgicos**, cuya principal diferencia es el tipo de proteína G con la que se acoplan (Go y Gs, respectivamente). Cada tipo principal de receptor adrenérgico incluye **varios subtipos** con pequeñas diferencias estructurales que afectan su función y distribución. Se conocen en total 9 subtipos: α1A, α1B, α1D, α2A, α2B, α2C, β1, β2 y β3. Cada uno de ellos está distribuidos en **múltiples tejidos y órganos**, y median **efectos fisiológicos muy diversos**. Por ejemplo, los α-adrenérgicos están generalmente presentes en músculo liso de vasos sanguíneos, por lo que su activación está asociada a vasoconstricción. Por su parte, los receptores β1 predominan en el músculo cardíaco, para regular su frecuencia y fuerza de contracción; mientras que los β2 abundan en tejido respiratorio, causando broncodilatación. 

Por todo esto, los RA han sido ampliamente estudiados, ya que representan **blancos terapéuticos muy relevantes para diseñar fármacos** destinados al tratamiento de diversas enfermedades, incluyendo hipertensión, hipotensión, arritmias, asma, etc. 

Sin embargo, la **alta homología estructural entre los subtipos** representa un desafío significativo en el diseño de fármacos, ya que son muy comunes los **efectos secundarios** indeseados por la falta de especificidad, requiriéndose un exhaustivo trabajo de diseño para alcanzar una adecuada selectividad por el subtipo correspondiente. Por lo tanto, diseñar ligandos altamente selectivos sigue siendo un objetivo central en farmacología adrenérgica.

Por ejemplo, una característica diferencial ampliamente conocida es que los **receptores β** poseen un subsitio extra en su sitio de unión, ausente en los receptores α, y que explica - entre otras cosas - por qué norepinefrina es mucho más activo frente a receptores α que a β. Esta es una de las diferencias clave que suele explotarse para aumentar afinidad por uno u otro receptor, como veremos más adelante. Para más información sobre esta familia terapéutica, pueden recurrir al capítulo 23 del libro "*An Introduction to Medicinal Chemistry*" de Patrick (disponible en la biblioteca virtual).

<img src="Alfa_Beta.png" alt="Figura 3" width="800"/>



## 🔍 Exploración de ligandos adrenérgicos en fase clínica


📌 **Objetivo**: Obtener todos los moduladores (agonistas y antagonistas) de los cuatro receptores adrenérgicos principales descriptos en humanos (*α1, α2, β1 y β2*), limitando la búsqueda a aquellos que se encuentran en fase 4 de desarrollo clínico (es decir, ya están aprobados o disponibles comercialmente). 

Asimismo, y a los fines de poder avanzar en la construcción del set de estudio, se ha sumado como criterio de inclusión que los modulares posean estructura química conocida (SMILES disponibles en ChEMBL), como así también un mecanismo de acción farmacológico definido, resultando en su clasificación como ***agonistas*** o ***antagonistas***.

La función que vamos a utilizar hace todo eso automáticamente y nos devuelve un único *DataFrame* con la siguiente información para cada compuesto:

* Su identificador ChEMBL ID,
* Su nombre comercial (si está disponible),
* Su estructura química (SMILES),
* El receptor en el que actúa,
* El tipo de acción (agonista o antagonista),



In [ ]:
receptores = {
    "A1": "CHEMBL2094251", # Corresponde al código de identificación en ChEMBL de del receptor α1 adernérgico
    "A2": "CHEMBL2095158", # Corresponde al código de identificación en ChEMBL de del receptor α2 adernérgico
    "B1": "CHEMBL213", # Corresponde al código de identificación en ChEMBL de del receptor β1 adernérgico
    "B2": "CHEMBL210" # Corresponde al código de identificación en ChEMBL de del receptor β2 adernérgico
}

df_ligandos = obtener_agonistas_antagonistas_adrenergicos(receptores)
itables.show(df_ligandos)

💾 **Grabar los resultados** de búsqueda al disco local puede resultar conveniente.

In [ ]:
#df_ligandos = pd.read_csv("agentes_adrenergicos_aprobados.csv") # Este comando resulta de utilidad si se require cargar rápidamente el df desde el disco
df_ligandos.to_csv("agentes_adrenergicos_aprobados.csv")

📸 **Visualizar** sus estructuras químicas puede ser muy útil. 

Para esto, vamos a utilizar la siguiente función que nos permite representar un conjunto de moléculas alineadas y organizadas por receptor, tipo de acción y nombre comercial.

Si queremos visualizar TODOS los resultados, la ejecutamos de la siguiente manera:

In [ ]:
# Mostrar ambos tipos de acción para todos los receptores
dibujar_moleculas(df_ligandos)

📸 También podríamos definir qué datos dibujar. Por ejemplo, solamente agonistas de todos los receptores.

In [ ]:
# Mostrar todos los agonistas 
dibujar_moleculas(df_ligandos, tipo_accion="AGONIST")

📸 O ser más específicos aún, y dibujar solamente aquellos con cierta actividad sobre blanco(s) en particular.

In [ ]:
# Mostrar todos los agonistas de B2
dibujar_moleculas(df_ligandos, receptores_seleccionados=["B2"], tipo_accion="AGONIST")

💡 Analice las siguientes estructuras, ¿identifican características comunes? ¿Cuál/es?

💡 Ejecute las siguientes funciones para comparar la estructura de epinefrina (prototipo) con el de algunos de los fármacos aprobados. ¿Qué estrategias de diseño logran identificar? ¿Con qué fin creen que se han aplicado?


In [ ]:
ids_moleculas = ["RIMITEROL", "ISOETHARINE HYDROCHLORIDE", "BITOLTEROL MESYLATE"]  
comparar_moleculas_con_template(df_ligandos, ids_moleculas, smiles_template="CNCC(O)c1ccccc1")

In [ ]:
ids_moleculas = ["FENOTEROL", "METAPROTERENOL SULFATE", "TERBUTALINE SULFATE", "REPROTEROL"]  
comparar_moleculas_con_template(df_ligandos, ids_moleculas, smiles_template="CNCC(O)c1ccccc1")

In [ ]:
ids_moleculas = ["ALBUTEROL", "PIRBUTEROL ACETATE", "CLENBUTEROL"]
comparar_moleculas_con_template(df_ligandos, ids_moleculas, smiles_template="CNCCO")

In [ ]:
ids_moleculas = ["ALBUTEROL", "ALBUTEROL SULFATE", "LEVALBUTEROL HYDROCHLORIDE", "LEVALBUTEROL TARTRATE"]
comparar_moleculas_con_template(df_ligandos, ids_moleculas, smiles_template="CNCCO")

📚 Desde el inicio del cuatrimestre, hemos enfatizado la importancia de las **propiedades** de los fármacos (o potenciales fármacos) y su relación con la estructura química y la actividad biológica. Hemos discutido, además, ciertos valores de referencia que la mayoría de los fármacos comparten. Estas características comunes, que definen la viabilidad de un compuesto como potencial fármaco, se agrupan bajo lo que se conoce como las propiedades **"drug-like"**.

Una de las teorías más comúnmente aplicadas en el proceso de diseño de fármacos es la de **Lipinski**, que establece una serie de reglas o límites para propiedades drug-like clave, como el **peso molecular** (<500), el **logP** (<5), la cantidad de **grupos donores** (<5) **y aceptores** (<10) **de puente hidrógeno**, y la **flexibilidad** (<10 enlaces rotables), que suelen ser características comunes en los fármacos orales efectivos. Según las reglas de Lipinski, un compuesto con más de uno de estos parámetros fuera de los rangos establecidos es menos probable que sea un buen candidato para ser un fármaco oral.

A continuación, analizaremos las propiedades de varios ligandos aprobados para su uso clínico y las compararemos con las propiedades "drug-like" propuestas por Lipinski, observando cómo se alinean con lo que hemos aprendido hasta ahora sobre diseño de fármacos

In [ ]:
df_ligandos = calcular_propiedades(df_ligandos)
itables.show(df_ligandos)

Para facilitar el análisis, utilizaremos una visualización que de conoce como de ***[violín](https://mode.com/blog/violin-plot-examples)***, y que permite apreciar de manera muy intuitiva la distribución de datos. Veámoslo gráficamente... ¿Cumplen con las reglas de Lipinski?

In [ ]:
graficar_distribucion_descriptores(df_ligandos)

Ya hemos analizado los fármacos adrenérgicos que se encuentran aprobados para su uso clínico. Sin embargo, como dijimos al inicio, en ChEMBL también podemos obtener información de todas las moléculas que han sido evaluadas frente a estos blancos terapéuticos, hayan llegado o no a fase clínica.

## 🔍 Exploración de ligandos adrenérgicos en fase exploratoria, preclínica y/o clínica


Comencemos por extraer todos los ligandos que posean algún tipo de reporte de actividad frente a los 4 receptores adrenérgicos, independientemente de si han avanzado o no hacia ensayos clínicos.

In [ ]:
df_adrenergicos = descargar_ligandos_adrenergicos(receptores, df_ligandos)
df_adrenergicos

💾 **Grabar los resultados** de búsqueda al disco local puede resultar conveniente dado que consume un tiempo significativo. El archivo generado puede ser recargado rápidamente si se requieren procesamientos adicionales.

In [ ]:
df_adrenergicos.to_csv("agentes_adrenergicos_todos.csv")
#df_adrenergicos = pd.read_csv("agentes_adrenergicos_todos.csv") # Este comando resulta de utilidad si se require cargar rápidamente el df desde el disco

🔬 Como se puede observar en la tabla, muchos ligandos pueden tener varios tipos de actividad biológica reportada (IC50, Ki, Kd, EC50, % de inhibición, etc.). 

Contemos entonces cuántos ligandos *únicos* hay reportado para cada receptor:

In [ ]:
contar_ligandos_unicos_por_receptor(df_adrenergicos)

Asimismo, pueden existir ligandos que hayan sido evaluados frente a varios receptores, con resultados de actividad biológica idéntica, similar o diferente. 

In [ ]:
df_pivot = actividad_por_receptor(df_adrenergicos, tipo_actividad="IC50")
itables.show(df_pivot)

#### **Espacio Químico**

En el ámbito de la química computacional y el diseño de fármacos, uno de los principales retos es comprender la relación entre la estructura molecular y sus propiedades. Una de las estrategias empleadas es visualizarlas en un **espacio químico**, en el que cada molécula se representa como un punto en un espacio multidimensional, basado en sus propiedades. Sin embargo, la representación de este espacio puede ser difícil de manejar debido a su alta dimensionalidad, especialmente cuando estamos trabajando con cientos o miles de compuestos.

Una de las técnicas más utilizadas para abordar este problema es la *reducción de dimensionalidad**, que nos permite proyectar este espacio químico multidimensional en una forma más comprensible y visualmente interpretable, manteniendo la mayor cantidad de información posible sobre las relaciones estructurales de las moléculas. Entre las herramientas más poderosas en este campo se encuentra **UMAP** (*Uniform Manifold Approximation and Projection*). UMAP es un algoritmo de reducción de dimensionalidad no lineal que se ha destacado por su capacidad para preservar las relaciones locales y globales en los datos, lo que lo hace ideal para representar estructuras químicas complejas.

En particular, el uso de UMAP permite representar un conjunto de estructuras de manera que **las moléculas químicamente similares se agrupen en el espacio reducido, mientras que las moléculas disímiles se separen**. Esto facilita la visualización de clusters o patrones que podrían no ser evidentes en la estructura original de los datos.

Para facilitar aún más la interpretación visual de estos datos, se han desarrollado herramientas como los **mapas químicos** o **chemplots**, que permiten representar gráficamente, intuitivamente y estéticamente agradables estos espacios químicos reducidos. 

Veamos un ejemplo de este tipo de mapas, aplicando la reducción de dimensionalidad mediante UMAP (puede llevar un par de minutos) a los ligandos adrenérgicos que filtramos, y finalmente graficar el resultado.

In [ ]:
df_adrenergicos_chemspace = determinar_chemspace(df_adrenergicos)
df_adrenergicos_chemspace

💾 **Grabar los resultados:** El cómputo de los valos de UMAP también puede demandar un tiempo considerable, razón por la cual también es recomendable guardar los resultados generados en el disco local por si se quisieran recargar rápidamente.

In [ ]:
df_adrenergicos_chemspace.to_csv("agentes_adrenergicos_chemspace.csv")
#df_adrenergicos_chemspace = pd.read_csv("agentes_adrenergicos_chemspace.csv") # Este comando resulta de utilidad si se require cargar rápidamente el df desde el disco

### 🎯 **Preparación de datos para QSAR: agonistas β2 como fármacos broncodilatadores**

Para poder llevar adelante un análisis de relaciones cuantitativas estructura-actividad (**QSAR**), es fundamental trabajar con un conjunto de moléculas que compartan ciertas características. Ello implica que los análogos que componen el set de trabajo deben tener afinidad por el **mismo** blanco terapéutico, actuar mediante un **mecanismo de acción homólogo**, presentar cierto grado de semejanza estructural entre sí, y contar con datos de actividad biológica medidos bajo condiciones experimentales comparables. Esto asegura que las variaciones en la actividad puedan atribuirse **exclusivamente** a las diferencias estructurales y no a otros factores externos.

Con este objetivo en mente, y enfocándonos en compuestos activos sobre el receptor adrenérgico β2, vamos a aislar un subconjunto representativo de moléculas que cumplan con estos criterios. Este conjunto será utilizado en el TP N°4 como base para construir y evaluar modelos QSAR, facilitando así la interpretación y la predicción de relaciones estructura-actividad en este sistema biológico de interés.

#### **Agonistas β₂-adrenérgicos**

Los **agonistas β₂-adrenérgicos** son fármacos de gran utilidad terapéutica, especialmente en el tratamiento del asma y otras afecciones respiratorias. Su acción se basa en la activación del receptor β₂-adrenérgico, lo que produce la relajación del músculo liso bronquial, desencadenando una dilatación de las vías respiratorias y facilitando así el proceso respiratorio.

En situaciones de emergencia, se ha utilizado ***adrenalina*** para lograr esta dilatación bronquial. Sin embargo, su uso prolongado no es adecuado debido a su corta duración de acción y a los efectos secundarios cardiovasculares que provoca. Durante un tiempo, se empleó también ***isoprenalina*** como agente antiasmático, pero este compuesto carecía de selectividad entre los distintos subtipos de receptores β, lo que implicaba una activación no deseada de los receptores β₁ en músculo cardíaco y, en consecuencia, efectos cardiovasculares adversos. 

Otros análogos desarrollados en esa época también presentaron limitaciones, principalmente por su corta duración de acción, ya que eran absorbidos rápidamente por los tejidos y metabolizados por la enzima catecol-O-metiltransferasa (COMT). Para superar este problema, se intentó modificar la estructura química de estos fármacos. Estas modificaciones estructurales llevaron al desarrollo de un compuesto mejorado: el ***salbutamol*** (conocido como albuterol en Estados Unidos). En esta molécula, el hidroxilo fenolico en posición meta fue reemplazado por un grupo hidroximetileno. Como resultado, se obtuvo un fármaco con la misma potencia que la isoprenalina, pero 2000 veces menos activo frente a β₁ y mucho más resistente a metabolización. Fue comercializado inicialmente como un racemato y se convirtió rápidamente en uno de los medicamentos más utilizados para tratar el asma.


🔎 Para aislar agonistas β₂-adrenérgicos reportados en ChEMBL, comencemos por separar ligandos con actividad frente a receptores beta de aquellos activos frente al receptor alfa.

In [ ]:
df_alfa, df_beta = separar_alfa_beta(df_adrenergicos_chemspace)

In [ ]:
df_alfa

In [ ]:
graficar_espacio_quimico_con_filtro(df_adrenergicos_chemspace, df_alfa)

In [ ]:
df_beta

In [ ]:
graficar_espacio_quimico_con_filtro(df_adrenergicos_chemspace, df_beta)

Y ahora, aislemos solamente a los que tienen actividad frente al receptor β2.

In [ ]:
df_beta1, df_beta2 = separar_beta1_beta2(df_beta)

In [ ]:
df_beta1

In [ ]:
graficar_espacio_quimico_con_filtro(df_adrenergicos_chemspace, df_beta1)

In [ ]:
df_beta2

In [ ]:
graficar_espacio_quimico_con_filtro(df_adrenergicos_chemspace, df_beta2)

Recordemos que uno de los requisitos fundamentales para realizar un análisis QSAR apropiado es que la **potencia o actividad biológica** de las moléculas esté **expresada en el mismo tipo de parámetro y en las mismas unidades**. Esto permite una comparación directa entre compuestos y asegura la coherencia de los modelos. Por esta razón, vamos a seleccionar exclusivamente aquellos ligandos del receptor adrenérgico β2 que tengan valores de IC50 reportados, ya que este es uno de los parámetros más comúnmente utilizados en la literatura para cuantificar actividad biológica.

In [ ]:
df_beta2_IC50 = aislar_actividad_biologica(df_beta2, tipo='IC50')
itables.show(df_beta2_IC50)

En la tabla pueden observar que algunos compuestos son en realidad totalmente inactivos, ya que su actividad biológica se encuentra reportada con el formato "potencia > X" (Revisar columna "*relation*").
Estos datos no deben formar parte de un modelo de **QSAR**, ya que **solamente se deben considerar moléculas activas**. Por lo tanto, eliminaremos esos datos antes de proseguir con el análisis.

In [ ]:
df_beta2_IC50 = df_beta2_IC50[df_beta2_IC50['relation'] == '=']
itables.show(df_beta2_IC50)

Ahora nos enfrentamos a un problema. Como vemos en la tabla, en la columna *action_type*, muy pocos ligandos tienen informado su mecanismo de acción frente al receptor β2 adrenérgico, lo que a priori dificultaría la identificación de moléculas que actúan como agonistas o antagonistas de este blanco terapéutico. Sin embargo, al comparar las estructuras químicas de ligandos con distinto mecanismo, podemos encontrar una diferencia crucial que nos ayudará a clasificarlos. 

💡Ejecuten las siguientes funciones e intenten descifrar dicha diferencia:

In [ ]:
#Agonistas β2 
ids_moleculas = ["ISOPROTERENOL", "ALBUTEROL","CLENBUTEROL"] 
comparar_moleculas_con_template(df_beta2_IC50, ids_moleculas, smiles_template="CNCCO")

In [ ]:
#Antagonistas β2 
ids_moleculas = ["TIMOLOL", "CARVEDILOL", "PROPRANOLOL HYDROCHLORIDE", "ATENOLOL"]
comparar_moleculas_con_template(df_beta2_IC50, ids_moleculas, smiles_template="CNCCO")

Efectivamente, está ampliamente reportado que los **agonistas β₂** activan el receptor provocando un cambio conformacional al formar un **puente de hidrógeno con el residuo Ser5.46**, lo que permite que el receptor adopte su forma activa. En cambio, los **antagonistas β₂** no forman ese puente de hidrógeno. En su lugar, ocupan ese espacio con grupos voluminosos e hidrofóbicos que bloquean el movimiento del receptor, impidiendo así que se active. Entonces, aprovechemos estas diferencias estructurales para filtrar ligandos y clasificarlos como agonistas o antagonistas del receptor β2 adrenérgico. 

<div align="center">
<img src="Ag_Antag.png" alt="Figura 4" width="800"/>
</div>



In [ ]:
df_beta2_antagonists = filtrar_por_grupo_funcional(df_beta2_IC50, smarts='OCC(O)CNC')
itables.show(df_beta2_antagonists)

In [ ]:
graficar_espacio_quimico_con_filtro(df_adrenergicos_chemspace, df_beta2_antagonists)

In [ ]:
df_beta2_agonists = filtrar_por_grupo_funcional(df_beta2_IC50, smarts='c1ccccc1C(O)CNC')
itables.show(df_beta2_agonists)

In [ ]:
graficar_espacio_quimico_con_filtro(df_adrenergicos_chemspace, df_beta2_agonists)

Al igual que hicimos previamente, podríamos analizar y comparar las propiedades fisicoquímicas de estos ligandos.

In [ ]:
df_beta2_antagonists = calcular_propiedades(df_beta2_antagonists)
itables.show(df_beta2_antagonists)

In [ ]:
ver_rangos_propiedades(df_beta2_antagonists)

In [ ]:
df_beta2_agonists = calcular_propiedades(df_beta2_agonists)
itables.show(df_beta2_agonists)

In [ ]:
ver_rangos_propiedades(df_beta2_agonists)

In [ ]:
graficar_distribucion_descriptores(df_beta2_agonists, "MolWt")

Como podemos observar, muchas moléculas poseen propiedades que exceden los límites típicamente asociados a fármacos ("drug-like"). Podríamos, entonces, filtrar solamente aquellos que sí cumplan con las reglas de Lipinski, y emplear así dicho set para el posterior estudio de QSAR (TP N°4).


In [ ]:
criterios = {
    'MolLogP': (0, 5),
    'MolWt': (0, 500),
    'NumHAcceptors': (0, 10),
    'NumHDonors': (0, 5),
    'NumRotatableBonds': (0, 10)
}

df_beta2_agonists_lipinski = filtrar_multiples_propiedades(df_beta2_agonists, criterios)
itables.show(df_beta2_agonists_lipinski)

In [ ]:
graficar_espacio_quimico_con_filtro(df_adrenergicos_chemspace, df_beta2_agonists_lipinski)

Si volvemos a analizar la tabla, podremos observar un detalle: a pesar de la diversidad de moléculas, la gran mayoría provienen de un mismo reporte bibliográfico (moléculas con el mismo *document_chembl_id*).
Esto es muy importante para los estudios de QSAR, ya que podemos confiar en que la determinación de la actividad biológica se llevó a cabo bajo condiciones experimentales idénticas o similares, y que muchas de estas moléculas serán análogas entre sí y obtenidas a partir de la aplicación de modificaciones estructurales racionales.

In [ ]:
itables.show(df_beta2_agonists_lipinski.sort_values(by = "document_chembl_id", ascending = False))

Uno de ellos llama especialmente la atención, ya que casi la mitad de los compuestos provienen de él: **CHEMBL3886528**. Si buscamos este documento en ChEMBL podemos identificar que se trata de una **patente** titulada "*Uso de fenoterol y análogos del fenoterol en el tratamiento de glioblastomas y astrocitomas*", en la que se reportan y protegen intelectualmente una serie de análogos estructuralmente similares como agonistas β2 adrenérgicos para el tratamiento de ciertos tipos de cáncer que sobreexpresan dicho receptor, como es el caso de algunos tumores cerebrales.

<div align="center">
<img src="Patent.png" alt="Figura 5" width="800"/>
<div/>
    
    
Tras leer y analizar en detalle dicho documento, notamos que algunas moléculas allí incluidas no tienen su IC50 reportado en ChEMBL, y por ese motivo no se sumaron al set de moléculas filtradas hasta este punto. Por lo tanto, hemos extraído esta información y la agregaremos al set final para poder establecer relaciones estructura-actividad más contundentes.

In [ ]:
df_b2_agonists_final = combinar_dataframes(df_beta2_agonists_lipinski, 'patent.csv')
itables.show(df_b2_agonists_final)

In [ ]:
ver_rangos_propiedades(df_b2_agonists_final)

Hasta ahora, veníamos analizando la distribución de cada descriptor molecular de forma individual. Sin embargo, ¿qué ocurre si queremos compararlos entre sí o utilizarlos juntos en un modelo matemático, como en un análisis QSAR?

In [ ]:
graficar_distribucion_descriptores_all(df_b2_agonists_final, "IC50_value_nM")

Como se puede observar en los gráficos, cada descriptor tiene un rango de valores muy diferente. Esta disparidad puede generar problemas al momento de construir modelos, ya que los descriptores con valores numéricamente mayores pueden tener una influencia desproporcionada en las ecuaciones, independientemente de su verdadera relevancia biológica o química. En otras palabras, el modelo podría "pensar" que un descriptor es más importante solo porque sus valores son más grandes en escala.

Para evitar este sesgo, utilizamos una técnica conocida como **normalización de datos**. Esta consiste en transformar los valores de cada descriptor para que compartan un mismo rango (por ejemplo, entre 0 y 1), conservando las diferencias relativas entre moléculas. De esta manera, todos los descriptores contribuyen en igualdad de condiciones al modelo QSAR.

A continuación, ejecutaremos las siguientes funciones para visualizar claramente la diferencia antes y después de normalizar los datos.

In [ ]:
df_b2_agonists_final_norm = normalizar_desde_columna(df_b2_agonists_final, "IC50_value_nM")
itables.show(df_b2_agonists_final_norm)

In [ ]:
graficar_distribucion_descriptores_all(df_b2_agonists_final_norm, "IC50_value_nM_norm")

In [ ]:
df_b2_agonists_final_norm.to_csv("df_b2_agonists_final_norm.csv", index=False) #Guardamos el archivo para usar la próxima clase

💡 Por último, analicemos las estructuras de este set de compuestos e intentemos establecer algunas **relaciones estructura-actividad (SAR)**, previo al TP de la próxima semana.

In [ ]:
mostrar_y_guardar_moleculas_alineadas(df=df_b2_agonists_final_norm, grupo_funcional_smiles='CC(O)CNCC', mols_per_image=25, mols_per_row=5)

In [ ]:
ids_de_interes = ["CHEMBL1160723", "CHEMBL434"] # Isomería
mostrar_moleculas_por_id(df_b2_agonists_final_norm, ids_de_interes, grupo_funcional_smiles="CNCCO")

In [ ]:
ids_de_interes = ["CHEMBL229620", "CHEMBL389902"] # Isomería
mostrar_moleculas_por_id(df_b2_agonists_final_norm, ids_de_interes, grupo_funcional_smiles="CNCCO")

In [ ]:
ids_de_interes = ["CHEMBL389629", "CHEMBL229388", "CHEMBL387825", "CHEMBL229390"] # Isomería
mostrar_moleculas_por_id(df_b2_agonists_final_norm, ids_de_interes, grupo_funcional_smiles="CNCCO")

In [ ]:
ids_de_interes = ["CHEMBL388570", "CHEMBL229476", "CHEMBL229477", "CHEMBL389390"] # Isomería
mostrar_moleculas_por_id(df_b2_agonists_final_norm, ids_de_interes, grupo_funcional_smiles="CNCCO")

In [ ]:
ids_de_interes = ["CHEMBL228992", "CHEMBL228996", "CHEMBL228998",  "CHEMBL228417"] # Isomería
mostrar_moleculas_por_id(df_b2_agonists_final_norm, ids_de_interes, grupo_funcional_smiles="CNCCO")

In [ ]:
ids_de_interes = ["CHEMBL4113713", "CHEMBL3909417", "CHEMBL4115254", "CHEMBL3903905"] # Isomería
mostrar_moleculas_por_id(df_b2_agonists_final_norm, ids_de_interes, grupo_funcional_smiles="CNCCO")

In [ ]:
ids_de_interes = ["CHEMBL1800961", "CHEMBL1800962"] # Isomería
mostrar_moleculas_por_id(df_b2_agonists_final_norm, ids_de_interes, grupo_funcional_smiles="CNCCO")

In [ ]:
ids_de_interes = ["CHEMBL1800963", "CHEMBL1800934"] # Isomería
mostrar_moleculas_por_id(df_b2_agonists_final_norm, ids_de_interes, grupo_funcional_smiles="CNCCO")

In [ ]:
ids_de_interes = ["CHEMBL4280606", "CHEMBL4279962", "CHEMBL4279531", "CHEMBL4285281"] # Restricción + Isomería
mostrar_moleculas_por_id(df_b2_agonists_final_norm, ids_de_interes, grupo_funcional_smiles="CC(O)CNCC")

In [ ]:
ids_de_interes = ["CHEMBL389629", "CHEMBL388570", "CHEMBL228992", "CHEMBL4113713"] # Sustituciones aromáticas
mostrar_moleculas_por_id(df_b2_agonists_final_norm, ids_de_interes, grupo_funcional_smiles="CNCCO")

In [ ]:
ids_de_interes = ["CHEMBL228998", "CHEMBL4108239"] # Sustituciones aromáticas
mostrar_moleculas_por_id(df_b2_agonists_final_norm, ids_de_interes, grupo_funcional_smiles="CNCCO")

In [ ]:
ids_de_interes = ["CHEMBL388570", "CHEMBL1800936", "CHEMBL229476", "CHEMBL1800960"] # Homología superior
mostrar_moleculas_por_id(df_b2_agonists_final_norm, ids_de_interes, grupo_funcional_smiles="CNCCO")

In [ ]:
ids_de_interes = ["CHEMBL228992", "CHEMBL3099659", "CHEMBL3099662"] # Homología superior
mostrar_moleculas_por_id(df_b2_agonists_final_norm, ids_de_interes, grupo_funcional_smiles="c1ccccc1CCNCCO")

In [ ]:
ids_de_interes = ["CHEMBL229476", "CHEMBL228996", "CHEMBL1800934"] # Homología superior
mostrar_moleculas_por_id(df_b2_agonists_final_norm, ids_de_interes, grupo_funcional_smiles="CNCCO")

In [ ]:
ids_de_interes = ["CHEMBL229614", "CHEMBL1800962"] # Orientaciones.
mostrar_moleculas_por_id(df_b2_agonists_final_norm, ids_de_interes, grupo_funcional_smiles="CNCCO")

In [ ]:
ids_de_interes = ["CHEMBL3747487", "CHEMBL3747244", "CHEMBL3746280"] # Tamaños, restricciones conformacionales, sustituciones, orientaciones.
mostrar_moleculas_por_id(df_b2_agonists_final_norm, ids_de_interes, grupo_funcional_smiles="CNCCO")